In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
!unzip "/content/drive/MyDrive/GPT2/Articles.csv.zip"

Archive:  /content/drive/MyDrive/GPT2/Articles.csv.zip
  inflating: Articles.csv            


In [ ]:
def cleaning(s):
    s = str(s)  # Ensure the input is a string
    s = re.sub(r'\s\W', ' ', s)  # Replace whitespace followed by a non-word character with a space
    s = re.sub(r'\W,\s', ' ', s)  # Replace a non-word character followed by a comma and whitespace with a space
    s = re.sub(r"\d+", "", s)  # Remove all digits
    s = re.sub(r'\s+', ' ', s)  # Replace multiple whitespace characters with a single space
    s = re.sub(r'[!@#$_]', '', s)  # Remove specific special characters (!, @, #, $, _)
    s = s.replace("co", "")  # Remove occurrences of the substring "co"
    s = s.replace("https", "")  # Remove occurrences of the substring "https"
    s = s.replace("[\w*", " ")  # Replace the literal string "[\w*" with a space
    return s  # Return the cleaned string


In [ ]:
df = pd.read_csv("/content/Articles.csv", encoding="ISO-8859-1")
df = df.dropna()

In [ ]:
df.sample(5)

,Article,Date,Heading,NewsType
1967,St. KITTS: Australia vice-captain David Warner...,6/13/2016,Australia batsman Warner has fractured finger,sports
285,ISLAMABAD: Prime Minister Nawaz Sharif has und...,9/11/2015,pm nawaz underscores need for enhancing expor,business
2409,BIRMINGHAM: Pakistan captain Misbah-ul-Haq pat...,8/5/2016,Misbah guides Pakistan into lead over England,sports
2630,strong>TEHRAN: Iran has imposed no restriction...,2/6/2017,Iran US sanctions stop American oil firms proj,business
1667,strong>South Africa fast bowling great Allan D...,4/27/2016,Australia eye SAfrica great Donald as bowling,sports


In [ ]:
text_data = open("/content/Articles.txt", 'w')
for idx, item in df.iterrows():
  article = cleaning(item["Article"])
  text_data.write(article)
text_data.close()

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
# Loading Dataset

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [ ]:
# Loading Data Collator

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [ ]:
# Setting parameters

train_file_path = "/content/Articles.txt"
model_name = 'gpt2'
output_dir = '/content/results'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 3.0
save_steps = 500

In [ ]:
# Training
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Step,Training Loss
500,3.689000
1000,3.418600
1500,3.171200
2000,3.121600
2500,2.982500
3000,2.954500
3500,2.867400
4000,2.841600
4500,2.790900
5000,2.783600


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
# Testing

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "/content/results"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:
sequence = input()
max_len = int(input())
generate_text(sequence, max_len)

oil price
20


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


oil price had fallen about percent in April as oil exporters sought to cut output and were waiting for
